# Data Preprocessing

Preprocessing the features set stored in S3 bucket for model training, including dropping unnecessary columns, filtering out frauds cases, and normalizations. 

## Import Packages

In [ ]:
pip install awswrangler

In [3]:
import boto3

import awswrangler as wr

import pandas as pd
import numpy as np

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt

## Read dataset from S3 Bucket

Read the unlabelled enhanced dataset from 2020-12-06 to 2021-01-31 stored in s3 bucket.

In [4]:
df_train= pd.concat([wr.s3.read_csv(path="s3://fraud-user-profile-sandbox/ATO_Features_V2/dataset_DEC2020.csv.gz"),
                     wr.s3.read_csv(path="s3://fraud-user-profile-sandbox/ATO_Features_V2/dataset_JAN2021.csv.gz")],
                     ignore_index=True)

Read all types of frauds cases and ato fraud cases stored in s3 bucket.

In [6]:
# all types of frauds cases
bucket_name = 'fraud-user-profile-sandbox/ATO_Features_V2'
data_key = 'all_fraud_data.csv'
data_location = 's3://{}/{}'.format(bucket_name, data_key)
df_all_frauds = pd.read_csv(data_location)

# ATO frauds cases
data_key = 'ato_fraud_data.csv'
data_location = 's3://{}/{}'.format(bucket_name, data_key)
df_ato_frauds = pd.read_csv(data_location)

fill the nulls with 0, which are due to lack of iOS/Android app data.

In [5]:
df_train.fillna(0, inplace=True)
df_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33164907 entries, 0 to 33164906
Data columns (total 38 columns):
 #   Column                                       Non-Null Count     Dtype  
---  ------                                       --------------     -----  
 0   CUSTOMER_ID                                  33164907 non-null  int64  
 1   GA_SESSIONS_DATE                             33164907 non-null  object 
 2   DINSTINCT_PREV_CITY_CNTS                     33164907 non-null  int64  
 3   DISTINCT_CITY_DAILY_CNTS                     33164907 non-null  int64  
 4   DISTINCT_NEW_CITY_CNTS                       33164907 non-null  int64  
 5   CITY_NOT_SET_FLAG                            33164907 non-null  int64  
 6   DISTINCT_PREV_PRODUCTS_CLICKED_CNTS          33164907 non-null  int64  
 7   DISTINCT_PREV_PRODUCTS_TYPE1_CLICKED_CNTS    33164907 non-null  int64  
 8   DISTINCT_PREV_PRODUCTS_TYPE2_CLICKED_CNTS    33164907 non-null  int64  
 9   DISTINCT_NEW_PRODUCT_CLICKED_CNTS

fill the nulls in frauds dataframes with 0.

In [7]:
df_all_frauds.fillna(0, inplace=True)
df_ato_frauds.fillna(0, inplace=True)

## Random Sampling 

The raw dataset from Dec 2020 to Jan 2021 contains 33,164,907 examples, which is more than enough more model training. In order to reduce training time, we randomly sample 20% of the raw dataset as our unprocessed training data.

In [8]:
_, df_train = train_test_split(df_train, test_size=0.2, random_state=42)

## Remove frauds case in training set

Previous dataset indicates highly imbalance of our dataset, so we employ unsupervised models, and train those models only on normal customers cases and here we remove all frauds cases and ato cases from the dataset.

In [9]:
# Merge df_enhanced_features with all_fraud_cases
df_train = pd.merge(left=df_train, 
                  right=df_all_frauds[['CUSTOMER_ID','GA_SESSIONS_DATE']],
                  on=['CUSTOMER_ID','GA_SESSIONS_DATE'], 
                  how='left',
                  indicator=True)  

# Remove all the fraud cases from the merged dataframe
df_train = df_train[df_train['_merge'] == 'left_only']
df_train = df_train.drop(columns='_merge')

In [10]:
# Further remove the ato fraud cases from the merged dataframe
df_train = pd.merge(left=df_train, 
                  right=df_ato_frauds[['CUSTOMER_ID','GA_SESSIONS_DATE']],
                  on=['CUSTOMER_ID','GA_SESSIONS_DATE'], 
                  how='left',
                  indicator=True)  

df_train = df_train[df_train['_merge'] == 'left_only']
df_train = df_train.drop(columns='_merge')

Drop 'Customer_ID' and 'GA_SESSIONS_DATE'   from the dataset

In [ ]:
df_train = df_train.drop(columns=['CUSTOMER_ID','GA_SESSIONS_DATE'])
df_all_frauds = df_all_frauds.drop(columns=['CUSTOMER_ID','GA_SESSIONS_DATE'])
df_ato_frauds = df_ato_frauds.drop(columns=['CUSTOMER_ID','GA_SESSIONS_DATE'])

df_ato_frauds.drop_duplicates(inplace=True)

## Standadization of datasets

Before We standardize the datasets, we take 5% smaples out of the training data as validation sets for model evaluation.

In [ ]:
x_train, x_val = train_test_split(x_train, test_size=0.05, random_state=42)

In [ ]:
scaler_p2 = StandardScaler()
scaler_p2.fit(x_train)

x_train = scaler_p2.transform(x_train)

x_test_ato = scaler_p2.transform(df_ato_frauds)
x_test_all = scaler_p2.transform(df_all_frauds)

In [ ]:
# Print the size of different datasets
print('Size of training example:', x_train.shape)
print('Size of validation example:', x_val.shape)
print('Size of all frauds example:', x_test_all.shape)
print('Size of ato frauds example:', x_test_ato.shape)

## Now we have the datasets for model training and testing.